In [1]:
%reload_ext autoreload
%autoreload 2

import os
import json
from pathlib import Path
import multiprocessing

import jax
import jax.numpy as jnp
import pandas as pd
import numpy as np
import numpyro
import numpyro.distributions as dist

from hbmep.config import Config
from hbmep.model import Baseline
from hbmep.model.utils import Site as site
from hbmep.utils.constants import RECTIFIED_LOGISTIC

PLATFORM = "cpu"
jax.config.update("jax_platforms", PLATFORM)
numpyro.set_platform(PLATFORM)

cpu_count = multiprocessing.cpu_count() - 2
numpyro.set_host_device_count(cpu_count)
numpyro.enable_x64()


In [2]:
src = "/home/vishu/data/hbmep-processed/J_RCML_000/ground"

df = pd.read_csv(os.path.join(src, "data.csv"))
mat = np.load(os.path.join(src, "mat.npy"))
auc_window = np.load(os.path.join(src, "auc_window.npy"))

f = open(os.path.join(src, "muscles_map.json"))
muscles_map = json.load(f)

muscles_map

{'auc_1': 'LADM',
 'auc_2': 'LBiceps',
 'auc_3': 'LBicepsFemoris',
 'auc_4': 'LDeltoid',
 'auc_5': 'LECR',
 'auc_6': 'LFCR',
 'auc_7': 'LTriceps',
 'auc_8': 'RBiceps'}

In [3]:
""" Filter """
sub = [2, 5]
mat = mat[..., np.array(sub) - 1]

muscles_map = {u: v for u, v in muscles_map.items() if int(u.split("_")[-1]) in sub}

""" Save """
dst = "/home/vishu/data/hbmep-processed/J_RCML_000/ground/2_5"

df.to_csv(os.path.join(dst, "data.csv"), index=False)

np.save(os.path.join(dst, "mat.npy"), mat)
np.save(os.path.join(dst, "auc_window.npy"), np.array(auc_window))

f = open(os.path.join(dst, "muscles_map.json"), "w")
f.write(json.dumps(muscles_map))
f.close;

In [4]:
root_path = Path(os.getcwd()).parent.parent.parent.absolute()
toml_path = os.path.join(root_path, "configs/ground_Biceps_ECR.toml")

config = Config(toml_path=toml_path)
config.BUILD_DIR = "/home/vishu/repos/hbmep-paper/reports/J_RCML_000/ground/2_5/nb"


2023-07-25 09:15:09,258 - hbmep.config - INFO - Verifying configuration ...
2023-07-25 09:15:09,258 - hbmep.config - INFO - Success!


In [5]:
class RectifiedLogistic(Baseline):
    LINK = RECTIFIED_LOGISTIC

    def __init__(self, config: Config):
        super(RectifiedLogistic, self).__init__(config=config)

    def _model(self, subject, features, intensity, response_obs=None):
        intensity = intensity.reshape(-1, 1)
        intensity = np.tile(intensity, (1, self.n_response))

        feature0 = features[0].reshape(-1,)

        n_data = intensity.shape[0]
        n_subject = np.unique(subject).shape[0]
        n_feature0 = np.unique(feature0).shape[0]

        with numpyro.plate(site.n_response, self.n_response, dim=-1):
            with numpyro.plate(site.n_subject, n_subject, dim=-2):
                """ Hyper-priors """
                mu_a = numpyro.sample(
                    site.mu_a,
                    dist.TruncatedNormal(150, 50, low=0)
                )
                sigma_a = numpyro.sample(site.sigma_a, dist.HalfNormal(50))

                sigma_b = numpyro.sample(site.sigma_b, dist.HalfNormal(0.1))

                sigma_L = numpyro.sample(site.sigma_L, dist.HalfNormal(0.05))
                sigma_H = numpyro.sample(site.sigma_H, dist.HalfNormal(5))
                sigma_v = numpyro.sample(site.sigma_v, dist.HalfNormal(10))

                with numpyro.plate("n_feature0", n_feature0, dim=-3):
                    """ Priors """
                    a = numpyro.sample(
                        site.a,
                        dist.TruncatedNormal(mu_a, sigma_a, low=0)
                    )
                    b = numpyro.sample(site.b, dist.HalfNormal(sigma_b))

                    L = numpyro.sample(site.L, dist.HalfNormal(sigma_L))
                    H = numpyro.sample(site.H, dist.HalfNormal(sigma_H))
                    v = numpyro.sample(site.v, dist.HalfNormal(sigma_v))

                    g_1 = numpyro.sample("g_1", dist.Exponential(1 / 100))
                    g_2 = numpyro.sample("g_2", dist.Exponential(1 / 100))

        """ Model """
        mu = numpyro.deterministic(
            site.mu,
            L[feature0, subject]
            + jnp.maximum(
                0,
                -1
                + (H[feature0, subject] + 1)
                / jnp.power(
                    1
                    + (jnp.power(1 + H[feature0, subject], v[feature0, subject]) - 1)
                    * jnp.exp(-b[feature0, subject] * (intensity - a[feature0, subject])),
                    1 / v[feature0, subject]
                )
            )
        )
        beta = numpyro.deterministic(
            site.beta,
            g_1[feature0, subject] + g_2[feature0, subject] * (1 / mu)
        )

        with numpyro.plate(site.data, n_data):
            return numpyro.sample(
                site.obs,
                dist.Gamma(concentration=mu * beta, rate=beta).to_event(1),
                obs=response_obs
            )


model = RectifiedLogistic(config=config)

In [6]:
df, encoder_dict = model.load()

2023-07-25 09:15:09,296 - hbmep.dataset.core - INFO - Artefacts will be stored here - /home/vishu/repos/hbmep-paper/reports/J_RCML_000/ground/2_5/nb
2023-07-25 09:15:09,296 - hbmep.dataset.core - INFO - Copied config to /home/vishu/repos/hbmep-paper/reports/J_RCML_000/ground/2_5/nb
2023-07-25 09:15:09,297 - hbmep.dataset.core - INFO - Reading data from /home/vishu/data/hbmep-processed/J_RCML_000/ground/2_5/data.csv ...
2023-07-25 09:15:09,305 - hbmep.dataset.core - INFO - Processing data ...
2023-07-25 09:15:09,306 - hbmep.utils.utils - INFO - func:load took: 0.01 sec


In [7]:
# df = pd.read_csv(model.csv_path)

# sub = ["amap01", "amap02", "amap03", "amap04"]
# sub += ["amap05", "amap06", "amap07", "amap08"]

# ind = df.participant.isin(sub)

# df = df[ind].copy()
# df.reset_index(drop=True, inplace=True)
# print(df.shape)

# df, encoder_dict = model.load(df=df)


In [8]:
mcmc, posterior_samples = model.run_inference(df=df)


2023-07-25 09:15:09,335 - hbmep.model.baseline - INFO - Running inference with rectified_logistic ...


  0%|          | 0/10000 [00:00<?, ?it/s]

  0%|          | 0/10000 [00:00<?, ?it/s]

  0%|          | 0/10000 [00:00<?, ?it/s]

  0%|          | 0/10000 [00:00<?, ?it/s]

2023-07-25 13:03:58,818 - hbmep.utils.utils - INFO - func:run_inference took: 3 hr and 48 min


In [9]:
mcmc.print_summary(prob=.95)



                mean       std    median      2.5%     97.5%     n_eff     r_hat
  H[0,0,0]      2.31      1.33      1.93      0.76      4.77    526.98      1.01
  H[0,0,1]      0.58      0.04      0.58      0.51      0.66    956.99      1.01
  H[0,1,0]      3.07      1.81      2.49      1.39      6.55     26.84      1.06
  H[0,1,1]      1.53      1.94      0.84      0.23      5.04    116.40      1.04
  H[0,2,0]      2.28      0.48      2.17      1.60      3.24     30.83      1.06
  H[0,2,1]      1.34      0.06      1.34      1.23      1.46    767.22      1.01
  H[0,3,0]      5.83      0.34      5.80      5.16      6.53    386.90      1.01
  H[0,3,1]     10.20      0.34     10.18      9.55     10.89    708.39      1.01
  H[0,4,0]     10.07      6.82      8.30      1.59     23.30    204.34      1.02
  H[0,4,1]     12.29      5.57     11.10      4.60     22.94    631.73      1.00
  H[0,5,0]      9.97      1.20      9.82      8.17     11.77    223.77      1.02
  H[0,5,1]     12.73      0

In [12]:
model.render_recruitment_curves(df=df, encoder_dict=encoder_dict, posterior_samples=posterior_samples)


2023-07-25 13:12:19,953 - hbmep.model.baseline - INFO - Rendering recruitment curves ...


2023-07-25 13:16:19,583 - hbmep.model.baseline - INFO - Saved to /home/vishu/repos/hbmep-paper/reports/J_RCML_000/ground/2_5/nb/recruitment_curves.pdf
2023-07-25 13:16:19,584 - hbmep.utils.utils - INFO - func:render_recruitment_curves took: 3 min and 59.64 sec


In [13]:
model.render_predictive_check(df=df, encoder_dict=encoder_dict, posterior_samples=posterior_samples)


2023-07-25 13:16:19,845 - hbmep.model.baseline - INFO - Rendering Posterior Predictive Check ...
2023-07-25 13:21:00,631 - hbmep.model.baseline - INFO - Saved to /home/vishu/repos/hbmep-paper/reports/J_RCML_000/ground/2_5/nb/posterior_predictive_check.pdf
2023-07-25 13:21:00,633 - hbmep.utils.utils - INFO - func:render_predictive_check took: 4 min and 40.79 sec


In [14]:
model.save(mcmc=mcmc)

2023-07-25 13:21:00,699 - hbmep.model.baseline - INFO - Saving inference data ...
2023-07-25 13:22:23,821 - hbmep.model.baseline - INFO - Saved to /home/vishu/repos/hbmep-paper/reports/J_RCML_000/ground/2_5/nb/mcmc.nc
2023-07-25 13:22:23,822 - hbmep.model.baseline - INFO - Rendering convergence diagnostics ...
2023-07-25 13:27:04,867 - hbmep.model.baseline - INFO - Saved to /home/vishu/repos/hbmep-paper/reports/J_RCML_000/ground/2_5/nb/diagnostics.csv
2023-07-25 13:27:04,868 - hbmep.model.baseline - INFO - Evaluating model ...
/home/vishu/repos/hbmep-paper/.venv/lib/python3.11/site-packages/arviz/stats/stats.py:1037: RuntimeWarning: overflow encountered in exp
  weights = 1 / np.exp(len_scale - len_scale[:, None]).sum(axis=1)
/home/vishu/repos/hbmep-paper/.venv/lib/python3.11/site-packages/numpy/core/_methods.py:49: RuntimeWarning: overflow encountered in reduce
  return umr_sum(a, axis, dtype, out, keepdims, initial, where)
/home/vishu/repos/hbmep-paper/.venv/lib/python3.11/site-packa